<a href="https://colab.research.google.com/github/shyamsparrow/Jan-G4---NLP-Chatbot/blob/main/Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Standard libraries
import re
import string
from unicodedata import normalize
from typing import List, Optional, Union, Callable

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, PunktSentenceTokenizer
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
#from names_dataset import NameDataset

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)

class PreProcessing:
  def __init__(self, convert_lowercase = False,
               remove_num=False,
               remove_whitespaces=False,
               remove_spl_char=False,
               remove_stopwords=False):
    self.convert_lowercase = convert_lowercase
    self.remove_num=remove_num
    self.remove_whitespaces=remove_whitespaces
    self.remove_spl_char=remove_spl_char
  
  def preprocess(self, text):

    if self.convert_lowercase:
      text = to_lower(text)

    if self.remove_num:
      text = remove_num_method(text)
    
    if self.remove_spl_char:
      text = remove_spl_char_method(text)
    
    if self.remove_whitespaces:
      text = self.remove_wspace(text)

    return text
  def to_lower(self,text):
    """ Convert input text to lower case """
    return text.lower()

  def remove_number(self,text):
    """ Remove number in the input text """
    processed_text = re.sub('\d+', '', text)
    return processed_text

  def remove_url(self,text):
    """ Remove url in the input text """
    return re.sub('(www|http)\S+', '', text)

  def remove_punctuation(self,text,punctuations: Optional[str] = None):
    """
    Removes all punctuations from a string, as defined by string.punctuation or a custom list.
    For reference, Python's string.punctuation is equivalent to '!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~'
    """
    if punctuations is None:
        punctuations = string.punctuation
    processed_text = text.translate(str.maketrans('', '', punctuations))
    return processed_text

  def remove_special_character(self,text,special_characters: Optional[str] = None):
    """ Removes special characters """
    if special_characters is None:
        # TODO: add more special characters
        special_characters = 'å¼«¥ª°©ð±§µæ¹¢³¿®ä£'
    processed_text = text.translate(str.maketrans('', '', special_characters))
    return processed_text

  def keep_alpha_numeric(self,text):
    """ Remove any character except alphanumeric characters """
    return ''.join(c for c in text if c.isalnum())

  def remove_whitespace(text, remove_duplicate_whitespace: bool = True):
    """ Removes leading, trailing, and (optionally) duplicated whitespace """
    if remove_duplicate_whitespace:
        return ' '.join(re.split('\s+', input_text.strip(), flags=re.UNICODE))
    return input_text.strip()



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile config.py
remove_num=True
remove_spl_char = True
remove_whitespaces = False

Overwriting config.py


In [ ]:
import config

In [ ]:
config.remove_whitespaces

False

In [ ]:
pp = PreProcessing(remove_whitespaces=config.remove_whitespaces)

pp.preprocess("this is a sample text    ")

'this is a sample text    '